# Runtime Benchmarking

1. Setup and build the LLRS project.
2. Configure the benchmark by modifying the `config/runtime-benchmarking/config.yml` file.
3. Run the following notebook to generate the benchmark results.

In [8]:
import os
import yaml
import numpy as np
import datetime
import time
import logging
import json
import sys
from pathlib import Path
import pickle
import pandas as pd
from pathlib import Path
pd.set_option('display.max_columns', None)
from tqdm.notebook import tqdm
import re
import datetime
import subprocess
import math

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir) + "/modules/runtime-benchmarking/python/controllers")
sys.path.insert(0, str(project_dir) + "/modules/runtime-benchmarking/python")

from operational_benchmarking_problem import OperationalBenchmarkingProblem
import benchmark_helpers as bh


 ### Configure Paths ###

In [9]:
config_file      = str(project_dir) + "/configs/runtime-benchmarking/config.yml"
problems_folder  = str(project_dir) + "/resources/runtime-benchmark-problems" 
solutions_folder = str(project_dir) + "/resources/runtime-benchmark-solutions" 
runtime_folder   = str(project_dir) + "/resources/runtime-benchmark-data" 
metrics_folder   = str(project_dir) + "/resources/runtime-benchmark-metrics" 
aod_ops_folder   = str(project_dir) + "/resources/runtime-benchmark-aod-ops"
executable_path  = str(project_dir) + "/bin/modules/runtime-benchmarking/run-bench" 
solver_wrapper_so_file = str(project_dir) + "/bin/modules/llrs-lib/modules/solver/libllrs-lib-solver-so.so"


In [10]:
#create folders if they do not exist
os.makedirs(problems_folder, exist_ok=True)
os.makedirs(solutions_folder, exist_ok=True)
os.makedirs(runtime_folder, exist_ok=True)
os.makedirs(metrics_folder, exist_ok=True)
os.makedirs(aod_ops_folder, exist_ok=True)

In [11]:
# setup logging
logging.basicConfig(filename=str(project_dir) + '/resources/logs/benchmark-log.txt', level=logging.DEBUG)

 ### Define The Reconfiguration problems from <span style="color:orange">**benchmark_params.yml**</span> ###

In [12]:
# user-defined params
with open(config_file, 'r') as file:
    benchmark_params = yaml.safe_load(file)

#----------VARIABLES---------- 
# benchmark settings
trial_selector    = -1
num_trials        = benchmark_params["num_trials_per_problem"]
num_repetitions   = benchmark_params["repetitions_per_trial"]
allow_deficit     = False 
loss              = True
seed              = 0
if "trial_selector" in benchmark_params.keys():
    trial_selector = benchmark_params["trial_selector"]
# problem definition variables default
problem_def_dict = benchmark_params['problem_definition']


# add default loss atom params to dict
if "loss_atom_params" in problem_def_dict:
    bh.augment_dict(problem_def_dict["loss_atom_params"], bh.default_loss_atom_params)
else:
    problem_def_dict["loss_atom_params"] = bh.default_loss_atom_params

# add restricted loss env params to dict
if "loss_env_params" in problem_def_dict:
    bh.augment_dict(problem_def_dict["loss_env_params"], bh.default_loss_env_params)
else:
    problem_def_dict["loss_env_params"] = bh.default_loss_env_params

# add geometry params to dict
if "geometry_params" in problem_def_dict:
    bh.augment_dict(problem_def_dict["geometry_params"], bh.default_geometry_params)
else:
    problem_def_dict["geometry_params"] = bh.default_geometry_params

# problem range definition
enable_problem_range = False
#-----------------------------


 - Create problem definition dictionaries and objects by iterating through problem_range entries

In [13]:
# define number of problems 
if(enable_problem_range):
    # get first set of indp variables and set num_problems to its length
    first_var_list = list(list(problem_range_dict.values())[0].values())[0]
    num_problems = len(first_var_list)
else:
    # if range is not enabled, set problem range to only be the base problem 
    problem_range_dict = {"problem_params" : {"Nt_y" : [problem_def_dict["problem_params"]["Nt_y"]]}} 
    num_problems = 1 


 - All problems are index by a UUID, stored in output_data/problems

In [14]:
# generate list of reconfiguration problems and corresponding problem def dicts
problem_def_dict_list = []
reconfig_problems = []
problem_uuids = []

# iterate over problem range
for pblm_idx in range(num_problems):
    print(f"creating problem {pblm_idx}")
    np.random.seed(seed)
    # change indp vars in new problem instance, add uuid and psf/coef path
    temp_problem_dict = bh.augment_problem_dict(problem_def_dict, problem_range_dict, pblm_idx)

    # generate problem object (with random initial configs)
    # TODO: put this line inside the operational benchmarking block instead of here
    temp_problem = bh.make_problem_object(temp_problem_dict, num_trials, allow_deficit, trial_selector)

    problem_uuids.append(temp_problem_dict["uuid"])
    
    # append objects and dicts to respective lists
    problem_def_dict_list.append(temp_problem_dict)
    reconfig_problems.append(temp_problem)
    
# TODO log this 
bh.save_full_problem_definitions(problem_def_dict_list, benchmark_params, reconfig_problems, problems_folder)


creating problem 0


AttributeError: 'NoneType' object has no attribute 'lower'

 ### Pre-solve all the problems and save the solutions of configurations over all trials/repetitions/cycles ###

In [15]:
problem_obp_sols = []
obp_metrics_arr = []
obp_aod_ops_arr = []

# for every problem, run a Monte Carlo simulation on each trial
for pblm_idx, (rp, problem_dict) in enumerate(zip(reconfig_problems, problem_def_dict_list)):
    print("_____________________________________")
    trial_sols = []
    trial_metrics = []
    trial_aod_ops = []
    loss_env_params = problem_def_dict["loss_env_params"]
    algorithm = problem_dict["problem_params"]["algorithm"]

    np.random.seed(seed)
    seeds = np.random.randint(low=0, high=100*len(rp), size=len(rp))

    #IN PROGRESS: inplace StaticArray creation
    #target_static_array = 

    for seed_i, trial in zip(seeds, rp):
        np.random.seed(seed_i)

        # initialize OBP
        trial_obp = OperationalBenchmarkingProblem(trial["initial"], trial["target"],
                                            num_repetitions, algorithm) 

        # use .cu to solve and then simulate loss, num_repetitions times
        trial_obp_sol, trial_obp_metrics, trial_obp_aod_ops = trial_obp.pre_solve(loss, 
                                                                                  loss_env_params["t_alpha"], 
                                                                                  loss_env_params["t_nu"], 
                                                                                  loss_env_params["t_latency"],
                                                                                  solver_wrapper_so_file) 

        trial_sols.append(trial_obp_sol)
        trial_metrics.append(trial_obp_metrics)
        trial_aod_ops.append(trial_obp_aod_ops)
    
    problem_obp_sols.append(trial_sols)
    obp_metrics_arr.append(trial_metrics)
    obp_aod_ops_arr.append(trial_aod_ops)
    print("running problem {}".format(pblm_idx))

# save solutions to "uuid.json" in solutions folder. Also used by runtime benchmarking.
bh.save_obp_sols(problem_def_dict_list, problem_obp_sols, solutions_folder)
bh.save_obp_sols(problem_def_dict_list, obp_metrics_arr, metrics_folder)
bh.save_obp_sols(problem_def_dict_list, obp_aod_ops_arr, aod_ops_folder)



 ### Run all problems and <span style="color:orange">**collect run-time data**</span> ###

In [16]:
current_exp_params = problem_def_dict_list[0]['experiment_params']

iteration_object_with_progress_bar = tqdm(problem_def_dict_list, 
                                          desc='Processing problems', 
                                          bar_format='{l_bar}{bar:3}{r_bar}',
                                          total=len(problem_def_dict_list))

# execute C with all different saved json files

subprocess.run(["cp", config_file, str(project_dir) + "/configs/llrs/runtime-temp.yml"])
for problem_def_dict in iteration_object_with_progress_bar:
    command = executable_path + " runtime-temp.yml " + problem_def_dict['uuid']
    print("running", command)
    os.system(command)
subprocess.run(["rm", str(project_dir) + "/configs/llrs/runtime-temp.yml"])


IndexError: list index out of range

 ### Format each problem solutions with its benchmark results as a dataframe, store in desired path ###

In [17]:
is_2D = bh.is_2D_problem(problem_def_dict)
cur_time = str(datetime.datetime.now())
base_path = str(project_dir) + "/resources/pickled_results/" + cur_time.replace(" ", "/").replace(":", "-").replace(".", "-")
os.makedirs(base_path, exist_ok = True)
cur_time = int(time.time())
for puuid in problem_uuids:
    formatted_data_path = f"{base_path}/{puuid}.pickle"
    bh.store_formatted_benchmark_data(
        experiment_id=puuid, 
        runtime_dir=runtime_folder, 
        metrics_dir=metrics_folder, 
        config_dir=solutions_folder, 
        aod_ops_dir=aod_ops_folder, 
        problem_def_dir=problems_folder, 
        output_path=formatted_data_path, 
        is_2D=is_2D)



TypeError: '>' not supported between instances of 'NoneType' and 'int'

# Table Generation 

In [18]:
def preprocess(path):
    with open(path, "rb") as f:
        prob = pickle.load(f)

        print(prob["data"].columns.values)

        null_value = pd.Series()
        # we should probably make the settings.h into a yaml so we don't have to hard code it here
        waveform_duration = 10e4
        # waveforms streamed in step IV (instead of V)
        prob["data"]["Total_Waveforms"] = ((prob["data"][prob["data"]["Cycle"] > 0])["num_nu_operations"] + \
                                          (prob["data"][prob["data"]["Cycle"] > 0])["num_alpha_operations"]).mean()
        prob["data"]["S-IP_Total"] = prob["data"].get("II-Deconvolution", null_value) + prob["data"].get("II-Threshold", null_value)
        prob["data"]["S-Pre-lookup"] = prob["data"].get("V-First-Lookup", null_value)
        prob["data"]["S-Pre-upload"] = prob["data"].get("V-First-Upload", null_value) + prob["data"].get("V-Second-Upload", null_value)
        prob["data"]["S-Pre-update"] = prob["data"].get("V-First-Update", null_value)
        prob["data"]["S-Latency_Total"] = prob["data"]["S-Pre-lookup"] + prob["data"]["S-Pre-upload"] + prob["data"]["S-Pre-update"]
        prob["data"]["S-Synth_Load_Total"] = prob["data"].get("IV-Translate", null_value) + prob["data"]["S-Latency_Total"]
        prob["data"]["S-Conc_Load_Stream"] = prob["data"].get("V-Load_Stream", null_value) - prob["data"]["S-Latency_Total"]
        prob["data"]["S-Theoretical_Stream"] = prob["data"]["Total_Waveforms"] * waveform_duration
        prob["data"]["S-Stream_Latency"] = prob["data"]["S-Conc_Load_Stream"] - prob["data"]["S-Theoretical_Stream"]
        prob["data"]["S-Total"] = prob["data"].get("I", null_value) + prob["data"]["S-IP_Total"] + prob["data"].get("III-Total", null_value) + prob["data"].get("V-Load_Stream", null_value) + prob["data"].get("IV-Translate", null_value)
    return prob


 ### Load Data ###

In [20]:
os.chdir(str(project_dir))
BENCHMARKING_FIGURES_PATH = "../../../figures/benchmarking/"
path_1D =  "/home/tqtraaqs1/LLRS/resources/pickled_results/2024-04-14/20-51-34-299566/6d823d09-284a-4425-8c87-8fa8826e7a37.pickle"
path_2D =  "/home/tqtraaqs1/LLRS/resources/pickled_results/2024-04-14/21-11-01-535629/0fec3a34-b1a3-4e7b-8d08-0fffd1b7abf3.pickle"
path_2D2 = "/home/tqtraaqs1/LLRS/resources/pickled_results/2024-04-14/21-11-01-535629/0fec3a34-b1a3-4e7b-8d08-0fffd1b7abf3.pickle"
full_pickle_1D = preprocess(path_1D)
full_pickle_1D["title"] = "Linear 1D"
print("1D id: ", full_pickle_1D['problem_definition']['uuid'])

print()

full_pickle_2D = preprocess(path_2D)
full_pickle_2D["title"] = "REDREC\_V2"
print("2D id: ", full_pickle_2D['problem_definition']['uuid'])


full_pickle_2D2 = preprocess(path_2D2)
full_pickle_2D2["title"] = "REDREC\_V2"
print("2D id: ", full_pickle_2D2['problem_definition']['uuid'])

# uncomment to replace runtime data with dummy data for clarity 

#full_pickle_1D["title"] = "FAKE"
#for col in full_pickle_1D["data"].columns:
#   full_pickle_1D["data"][col].values[:] = 0

#full_pickle_2D["title"] = "FAKE"
#for col in full_pickle_2D["data"].columns:
#     full_pickle_2D["data"][col].values[:] = 0


['Trial' 'Repetition' 'Cycle' 'I' 'II-Deconvolution' 'II-Threshold'
 'III-Batching' 'III-Matching' 'III-Total' 'IV-Translate' 'V-First-Lookup'
 'V-First-Update' 'V-First-Upload' 'V-Latency' 'V-Load_Stream'
 'V-Second-Lookup' 'V-Second-Upload'
 'num_elementary_alpha_operations_atoms'
 'num_elementary_nu_operations_atoms' 'num_high_corruption_target_atoms'
 'num_atoms' 'num_nu_operations' 'num_elementary_nu_operations'
 'num_alpha_operations' 'num_elementary_alpha_operations' 'aod_operations'
 'config']
1D id:  6d823d09-284a-4425-8c87-8fa8826e7a37

['Trial' 'Repetition' 'Cycle' 'I' 'II-Deconvolution' 'II-Threshold'
 'III-Batching' 'III-Matching' 'III-Total' 'IV-Translate' 'V-First-Lookup'
 'V-First-Update' 'V-First-Upload' 'V-Latency' 'V-Load_Stream'
 'V-Second-Lookup' 'V-Second-Upload'
 'num_elementary_alpha_operations_atoms'
 'num_elementary_nu_operations_atoms' 'num_high_corruption_target_atoms'
 'num_atoms' 'num_nu_operations' 'num_elementary_nu_operations'
 'num_alpha_operations' 'n

/tmp/ipykernel_65230/3058257169.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  null_value = pd.Series()
/tmp/ipykernel_65230/3058257169.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  null_value = pd.Series()


['Trial' 'Repetition' 'Cycle' 'I' 'II-Deconvolution' 'II-Threshold'
 'III-Batching' 'III-Matching' 'III-Total' 'IV-Translate' 'V-First-Lookup'
 'V-First-Update' 'V-First-Upload' 'V-Latency' 'V-Load_Stream'
 'V-Second-Lookup' 'V-Second-Upload'
 'num_elementary_alpha_operations_atoms'
 'num_elementary_nu_operations_atoms' 'num_high_corruption_target_atoms'
 'num_atoms' 'num_nu_operations' 'num_elementary_nu_operations'
 'num_alpha_operations' 'num_elementary_alpha_operations' 'aod_operations'
 'config']
2D id:  0fec3a34-b1a3-4e7b-8d08-0fffd1b7abf3


/tmp/ipykernel_65230/3058257169.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  null_value = pd.Series()


In [21]:
print(full_pickle_1D.keys())
print(full_pickle_1D['problem_definition'].keys())
print(full_pickle_1D['problem_definition']['problem_params'].keys())
print(full_pickle_1D['problem_definition']['experiment_params'].keys())



dict_keys(['num_trials_per_problem', 'repetitions_per_trial', 'problem_definition', 'enable_problem_range', 'data', 'title'])
dict_keys(['experiment_params', 'problem_params', 'loss_atom_params', 'loss_env_params', 'geometry_params', 'uuid'])
dict_keys(['Nt_x', 'Nt_y', 'target_config_label', 'array_geometry_type', 'load_efficiency', 'target_config', 'num_target', 'algorithm'])
dict_keys(['roi_width', 'roi_height', 'detection_threshold', 'psf', 'coef_x', 'coef_y'])


In [22]:
{**full_pickle_1D, 'problem_definition' : {**full_pickle_1D['problem_definition'],  'problem_params': 
 {**full_pickle_1D['problem_definition']['problem_params'], 
  'target_config':str(full_pickle_1D['problem_definition']['problem_params']['target_config'])}}}


{'num_trials_per_problem': 1000,
 'repetitions_per_trial': 1,
 'problem_definition': {'experiment_params': {'roi_width': 16,
   'roi_height': 1024,
   'detection_threshold': 800,
   'psf': 'psfs.bin',
   'coef_x': 'default.csv',
   'coef_y': 'default.csv'},
  'problem_params': {'Nt_x': 1,
   'Nt_y': 64,
   'target_config_label': 'centre_compact',
   'array_geometry_type': 'rectangular lattice',
   'load_efficiency': 0.6,
   'target_config': '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]',
   'num_target': 32,
   'algorithm': 'LINEAR-EXACT-V2-1D'},
  'loss_atom_params': {'p_nu': 0.985, 'p_alpha': 0.985, 't_lifetime': 60},
  'loss_env_params': {'t_alpha': 1e-05, 't_nu': 1e-05, 't_latency': 0.02},
  'geometry_params': {'trap_number': [1, 64],
   'norms': 2e-06,
   'sampling_index': [],
   'z': 0,
   'shift_vector': [0, 0, 0],
   'k1': [1, 0, 0]

In [23]:
{**full_pickle_2D, 'problem_definition' : {**full_pickle_2D['problem_definition'],  'problem_params': 
 {**full_pickle_2D['problem_definition']['problem_params'], 
  'target_config':str(full_pickle_2D['problem_definition']['problem_params']['target_config'])}}}


{'num_trials_per_problem': 1000,
 'repetitions_per_trial': 1,
 'problem_definition': {'experiment_params': {'roi_width': 128,
   'roi_height': 1024,
   'detection_threshold': 800,
   'psf': 'psfs.bin',
   'coef_x': 'default.csv',
   'coef_y': 'default.csv'},
  'problem_params': {'Nt_x': 8,
   'Nt_y': 64,
   'target_config_label': 'centre_compact',
   'array_geometry_type': 'rectangular lattice',
   'load_efficiency': 0.6,
   'target_config': '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [24]:
print(full_pickle_1D["title"])
full_pickle_1D["data"].describe().drop('count')


Linear 1D


,Trial,Repetition,Cycle,I,II-Deconvolution,II-Threshold,III-Batching,III-Matching,III-Total,IV-Translate,V-First-Lookup,V-First-Update,V-First-Upload,V-Latency,V-Load_Stream,V-Second-Lookup,V-Second-Upload,num_high_corruption_target_atoms,num_atoms,num_nu_operations,num_elementary_nu_operations,num_alpha_operations,num_elementary_alpha_operations,Total_Waveforms,S-IP_Total,S-Pre-lookup,S-Pre-upload,S-Pre-update,S-Latency_Total,S-Synth_Load_Total,S-Conc_Load_Stream,S-Theoretical_Stream,S-Stream_Latency,S-Total
mean,498.297592,0.0,1.173034,463.105365,350.018953,342.738837,0.0,2554.583531,2554.583531,9537.329863,7.563405e+06,10009.194983,7.707928e+06,1.876781e+07,2.677108e+07,7.549426e+06,3.466069e+06,7.366934,36.876083,33.088925,70.344141,0.0,0.0,4.873381e+01,692.757790,7.563405e+06,1.117400e+07,10009.194983,1.874741e+07,1.875695e+07,8.023666e+06,4.873381e+06,3.150286e+06,2.678432e+07
std,289.483537,0.0,1.058612,165.969700,40.733158,24.819378,0.0,670.123256,670.123256,4642.812858,3.704744e+05,1909.480476,2.100993e+05,2.897790e+06,6.482029e+06,2.072780e+05,2.826343e+06,9.677484,4.016111,37.520435,81.059207,0.0,0.0,1.208117e-12,58.644334,3.704744e+05,2.838481e+06,1909.480476,2.895596e+06,2.895800e+06,3.942458e+06,2.552234e-07,3.942458e+06,6.482390e+06
min,0.000000,0.0,0.000000,321.000000,310.000000,310.000000,0.0,712.000000,712.000000,7635.000000,5.123274e+06,5921.000000,5.846966e+06,1.450333e+07,2.036837e+07,5.680229e+06,1.414672e+06,0.000000,25.000000,0.000000,0.000000,0.0,0.0,4.873381e+01,631.000000,5.123274e+06,7.458271e+06,5921.000000,1.448657e+07,1.449590e+07,5.859369e+06,4.873381e+06,9.859884e+05,2.038004e+07
25%,245.000000,0.0,0.000000,400.000000,331.000000,331.000000,0.0,2325.000000,2325.000000,8746.000000,7.494179e+06,7935.000000,7.653942e+06,1.681022e+07,2.267731e+07,7.499589e+06,1.487610e+06,0.000000,34.000000,0.000000,0.000000,0.0,0.0,4.873381e+01,671.000000,7.494179e+06,9.235422e+06,7935.000000,1.679300e+07,1.680153e+07,5.881723e+06,4.873381e+06,1.008342e+06,2.268968e+07
50%,499.000000,0.0,1.000000,420.000000,341.000000,341.000000,0.0,2605.000000,2605.000000,9197.000000,7.566006e+06,10891.000000,7.723695e+06,1.694320e+07,2.281319e+07,7.565233e+06,1.615594e+06,2.000000,37.000000,17.000000,33.000000,0.0,0.0,4.873381e+01,681.000000,7.566006e+06,9.342674e+06,10891.000000,1.692442e+07,1.693411e+07,5.892551e+06,4.873381e+06,1.019170e+06,2.282597e+07
75%,750.000000,0.0,2.000000,471.000000,351.000000,350.000000,0.0,2925.000000,2925.000000,9738.000000,7.642581e+06,11382.000000,7.793847e+06,2.277447e+07,3.269871e+07,7.632832e+06,7.573529e+06,17.000000,40.000000,55.000000,135.500000,0.0,0.0,4.873381e+01,692.000000,7.642581e+06,1.525335e+07,11382.000000,2.275309e+07,2.276221e+07,9.846027e+06,4.873381e+06,4.972646e+06,3.271162e+07
max,999.000000,0.0,6.000000,5360.000000,772.000000,701.000000,0.0,15289.000000,15289.000000,185962.000000,1.364335e+07,24757.000000,1.072541e+07,3.096057e+07,5.354954e+07,1.045027e+07,8.777994e+06,32.000000,52.000000,185.000000,340.000000,0.0,0.0,4.873381e+01,1422.000000,1.364335e+07,1.836368e+07,24757.000000,3.089561e+07,3.090559e+07,3.021971e+07,4.873381e+06,2.534633e+07,5.356391e+07


In [25]:
full_pickle_2D["data"].describe().drop('count')



,Trial,Repetition,Cycle,I,II-Deconvolution,II-Threshold,III-Batching,III-Matching,III-Total,IV-Translate,V-First-Lookup,V-First-Update,V-First-Upload,V-Latency,V-Load_Stream,V-Second-Lookup,V-Second-Upload,num_high_corruption_target_atoms,num_atoms,num_nu_operations,num_elementary_nu_operations,num_alpha_operations,num_elementary_alpha_operations,Total_Waveforms,S-IP_Total,S-Pre-lookup,S-Pre-upload,S-Pre-update,S-Latency_Total,S-Synth_Load_Total,S-Conc_Load_Stream,S-Theoretical_Stream,S-Stream_Latency,S-Total
mean,497.622373,0.0,1.275229,616.145988,363.177080,350.153687,42587.584350,6.860048e+05,7.285924e+05,21598.86117,7.520225e+06,8870.247371,7.539035e+06,1.840550e+07,2.160682e+07,7.392064e+06,3.317980e+06,9.465226,298.837526,17.184670,49.072211,10.481799,325.519976,3.929592e+01,713.330767,7.520225e+06,1.085702e+07,8870.247371,1.838611e+07,1.840771e+07,3.220704e+06,3.929592e+06,-7.088880e+05,2.235840e+07
std,289.535170,0.0,1.094693,681.699810,46.251675,21.615728,17352.459058,2.892917e+05,2.923773e+05,8130.37826,2.154074e+05,1607.478904,3.377093e+05,2.918290e+06,4.604905e+06,3.354809e+05,2.764254e+06,12.958291,12.372043,17.501914,60.160636,12.351182,386.258809,2.359351e-12,58.802846,2.154074e+05,2.872249e+06,1607.478904,2.915361e+06,2.915314e+06,2.071475e+06,2.673291e-07,2.071475e+06,4.650449e+06
min,0.000000,0.0,0.000000,371.000000,301.000000,300.000000,14087.000000,5.955440e+05,6.112140e+05,8045.00000,5.999577e+06,5591.000000,5.955627e+06,1.402476e+07,1.549113e+07,5.813927e+06,1.410100e+06,0.000000,259.000000,0.000000,0.000000,0.000000,0.000000,3.929592e+01,611.000000,5.999577e+06,7.444931e+06,5591.000000,1.400852e+07,1.403617e+07,3.597450e+05,3.929592e+06,-3.569847e+06,1.622936e+07
25%,246.000000,0.0,0.000000,461.000000,340.000000,341.000000,27162.000000,6.493350e+05,6.881060e+05,15409.00000,7.446171e+06,7775.000000,7.461920e+06,1.644190e+07,1.768106e+07,7.318473e+06,1.445761e+06,0.000000,291.000000,0.000000,0.000000,0.000000,0.000000,3.929592e+01,681.000000,7.446171e+06,8.963838e+06,7775.000000,1.642564e+07,1.644853e+07,1.260905e+06,3.929592e+06,-2.668687e+06,1.837791e+07
50%,496.000000,0.0,1.000000,501.000000,350.000000,350.000000,37201.000000,6.812220e+05,7.171480e+05,25238.00000,7.511305e+06,8436.000000,7.573877e+06,1.663796e+07,1.935128e+07,7.427511e+06,1.480100e+06,2.000000,299.000000,11.000000,18.000000,4.000000,120.000000,3.929592e+01,692.000000,7.511305e+06,9.107561e+06,8436.000000,1.661825e+07,1.663977e+07,2.730430e+06,3.929592e+06,-1.199162e+06,2.005739e+07
75%,750.000000,0.0,2.000000,661.000000,381.000000,360.000000,59643.000000,7.097180e+05,7.637850e+05,26680.00000,7.577762e+06,10059.000000,7.719311e+06,2.231906e+07,2.733042e+07,7.569926e+06,7.267728e+06,22.000000,307.000000,34.000000,102.000000,26.000000,816.000000,3.929592e+01,741.000000,7.577762e+06,1.477400e+07,10059.000000,2.229679e+07,2.232377e+07,5.015448e+06,3.929592e+06,1.085856e+06,2.816460e+07
max,999.000000,0.0,6.000000,37681.000000,962.000000,762.000000,104178.000000,1.464078e+07,1.473204e+07,206614.00000,1.120076e+07,23835.000000,1.136471e+07,2.965290e+07,3.720999e+07,1.119201e+07,1.055106e+07,49.000000,338.000000,66.000000,248.000000,32.000000,1020.000000,3.929592e+01,1593.000000,1.120076e+07,2.191577e+07,23835.000000,2.960066e+07,2.962583e+07,1.441296e+07,3.929592e+06,1.048336e+07,3.806193e+07


In [26]:
# print mean and median and std dev for relevant columns
ns_per_ms = 1e6

titles = ["I. Image Acquisition",
          "II.1 Deconvolution",
          "II.2 Thresholding",
          "III.1 Generate matching",
          "III.2 Batching",
          "IV.1 Generate table keys",
          "IV-V.1 Pre-lookup",
          "IV-V.2 Pre-upload",
          "IV-V.3 Pre-update",
          "IV-V.4 Waveform Synthesis & Streaming"]

stages = ["Image Acquisition",
          "Image Processing",
          "Problem Solving",
          "Waveform Synthesis & Streaming"]

sub_stages_list = [["I"],
              ["II-Deconvolution",
               "II-Threshold",
               "S-IP_Total"],
              ["III-Matching",
               "III-Batching",
               "III-Total"],
              ["IV-Translate",
               "V-First-Lookup",
               "V-First-Upload",
               "V-Second-Upload",
               "V-First-Update",
               "V-Load_Stream",
               "V-Latency",
               "S-Pre-lookup",
               "S-Pre-upload",
               "S-Pre-update",
               "S-Latency_Total",
               "S-Synth_Load_Total",
               "S-Conc_Load_Stream"]]

def print_runtime_summary(full_pickle):
    
    Nt_x = full_pickle["problem_definition"]["problem_params"]["Nt_x"]
    Nt_y = full_pickle["problem_definition"]["problem_params"]["Nt_y"]
    num_target = full_pickle["problem_definition"]["problem_params"]["num_target"]
    num_traps = Nt_x * Nt_y
    roi_height = full_pickle["problem_definition"]["experiment_params"]["roi_height"]
    roi_width = full_pickle["problem_definition"]["experiment_params"]["roi_width"]

    print("FORMAT: Component: Mean, Median(Std Dev)")
    print(full_pickle["title"])
    print(f"N_a = {num_target}, N_t = {num_traps}")
    print(f"ROI = {roi_height} X {roi_width}")
    
    runtime_data_dict = full_pickle["data"]
    for (stage, sub_stages) in zip(stages, sub_stages_list):
        print(f"------{stage}------")
        for sub_stage in sub_stages:
            value = runtime_data_dict.get(sub_stage, pd.Series())
            print(f"{sub_stage}: {value.mean()/ns_per_ms}, {value.median()/ns_per_ms}({value.std()/ns_per_ms})")
    print()


In [27]:

def theoretical_acquistion_time_aug21(T_exposure=0.02, f_v=1/4.33e-6, f_h=30e6, f_PCIe=1.3875e9, f_link=6.1e9, S_y=1039, 
                                      N_D=468, N_G=604, N_O=16, W=1024, H=1024, S_h=1056, bin_v=1, DMA_Const = 0.289e-3):
    print(H)

    T_transfer = S_y/f_v + H/f_v * sum([1/(2**i)for i in range(1, int(np.log2(bin_v)))])
    T_horshift = (H)/bin_v * (1/f_v + (S_h + N_O - 1024 + W)/f_h)
    T_register = (N_D + N_G)/f_h
    #T_link = 8 * W * H/bin_v / f_link
    T_link = 0
    T_dma = DMA_Const +H/bin_v * W * 2 / f_PCIe

    res = {"T_exposure" : T_exposure * 1e3,
           "T_transfer" : T_transfer * 1e3,
           "T_horshift" : T_horshift * 1e3,
           "T_register" : T_register * 1e3,
           "T_link" : 0,#T_link,
           "T_dma" : T_dma * 1e3,
           "T_total" : (T_exposure + T_transfer + T_horshift + T_register + T_link+ T_dma) * 1e3}
    print(W)
    print(res)

    return res

def compile_latex(latex_code, output_path):
    with open('temp.tex', 'w') as file:
        file.write(latex_code)

    # Compile the LaTeX code into a PDF file
    run(['pdflatex', 'temp.tex'])
    run(['mv', 'temp.pdf', output_path])

def replace_variables_in_template(template_path, variables):
    with open(template_path, 'r') as file:
        content = file.read()

    # Replace placeholders with their corresponding values
    for key, value in variables.items():
        content = content.replace('##' + key + '##', str(value))

    return content

template_path = str(project_dir) + '/modules/runtime-benchmarking/table_template.tex'
output_filename = f"runtime_table_{datetime.datetime.now()}.pdf"
output_path = f"{BENCHMARKING_FIGURES_PATH}/{output_filename}"

IA_pred_dict_1D = theoretical_acquistion_time_aug21(W=full_pickle_1D["problem_definition"]["experiment_params"]["roi_width"], H=full_pickle_1D["problem_definition"]["experiment_params"]["roi_height"])
IA_pred_dict_2D = theoretical_acquistion_time_aug21(W=full_pickle_2D["problem_definition"]["experiment_params"]["roi_width"] ,H=full_pickle_2D["problem_definition"]["experiment_params"]["roi_height"])
IA_pred_dict_2D2 = theoretical_acquistion_time_aug21(W=full_pickle_2D2["problem_definition"]["experiment_params"]["roi_width"] ,H=full_pickle_2D2["problem_definition"]["experiment_params"]["roi_height"])

#print(IA_pred_dict_2D["T_horshift"], IA_pred_dict_2D["T_register"])

#print(full_pickle_1D["problem_definition"]["problem_params"]["Nt_x"])
#print(full_pickle_2D["problem_definition"]["problem_params"]["Nt_x"])

# Variables to be inserted into the LaTeX template
variables = {
    "1D_path" : path_1D,
    "1D_title": full_pickle_1D["title"],
    "1D_Nt_x": full_pickle_1D["problem_definition"]["problem_params"]["Nt_x"],
    "1D_Nt_y" : full_pickle_1D["problem_definition"]["problem_params"]["Nt_y"],
    "1D_N_a1" : 1,
    "1D_N_a2" : (full_pickle_1D["problem_definition"]["problem_params"]["num_target"]),
    "1D_Nt1" : 1,
    "1D_Nt2" : full_pickle_1D["problem_definition"]["problem_params"]["Nt_y"],                
    "1D_roi_height" : full_pickle_1D["problem_definition"]["experiment_params"]["roi_height"],
    "1D_roi_width" : full_pickle_1D["problem_definition"]["experiment_params"]["roi_width"],
    "1D_IA_total_avg" : "{:.3f}".format(round(full_pickle_1D["data"]["I"].mean()/ns_per_ms, 3)),
    "1D_IA_total_std" : "{:.3f}".format(round(full_pickle_1D["data"]["I"].std()/ns_per_ms, 3)),
    "1D_IA_exposure" : "{:.3f}".format(round(IA_pred_dict_1D["T_exposure"])),
    "1D_IA_frame_transfer" : "{:.3f}".format(round(IA_pred_dict_1D["T_transfer"], 3)),
    "1D_IA_horz_shift" : "{:.3f}".format(round(IA_pred_dict_1D["T_horshift"] + IA_pred_dict_1D["T_register"], 3)),
    "1D_IA_DMA_transfer" : "{:.3f}".format(round(IA_pred_dict_1D["T_dma"] + full_pickle_1D["data"]["I"].mean()/ns_per_ms - IA_pred_dict_1D["T_total"], 3)),
    "1D_IA_latency" : "{:.3f}".format(round((full_pickle_1D["data"]["I"].mean()/ns_per_ms) - IA_pred_dict_1D["T_total"], 3)),
    "1D_IP_total_avg" : "{:.3f}".format(round(full_pickle_1D["data"]["S-IP_Total"].mean()/ns_per_ms, 3)),
    "1D_IP_total_std" : "{:.3f}".format(round(full_pickle_1D["data"]["S-IP_Total"].std()/ns_per_ms, 3)),
    "1D_IP_deconvolution" : "{:.3f}".format(round(full_pickle_1D["data"]["II-Deconvolution"].mean()/ns_per_ms, 3)),
    "1D_IP_threshold" : "{:.3f}".format(round(full_pickle_1D["data"]["II-Threshold"].mean()/ns_per_ms, 3)),
    "1D_PS_total_avg" : "{:.3f}".format(round(full_pickle_1D["data"]["III-Total"].mean()/ns_per_ms, 3)),
    "1D_PS_total_std" : "{:.3f}".format(round(full_pickle_1D["data"]["III-Total"].std()/ns_per_ms, 3)),
    "1D_PS_matching" : "{:.3f}".format(round(full_pickle_1D["data"]["III-Matching"].mean()/ns_per_ms, 3)),
    "1D_PS_batching" : "{:.3f}".format(round(full_pickle_1D["data"]["III-Batching"].mean()/ns_per_ms, 3)),
    "1D_SS_total_avg": "{:.3f}".format(round(full_pickle_1D["data"]["S-Synth_Load_Total"].mean()/ns_per_ms, 3)),
    "1D_SS_total_std": "{:.3f}".format(round(full_pickle_1D["data"]["S-Synth_Load_Total"].std()/ns_per_ms, 3)),
    "1D_SS_translate": "{:.3f}".format(round(full_pickle_1D["data"]["IV-Translate"].mean()/ns_per_ms, 3)),
    "1D_SS_Pre-lookup" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Pre-lookup"].mean()/ns_per_ms, 3)),
    "1D_SS_Pre-upload" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Pre-upload"].mean()/ns_per_ms, 3)),
    "1D_SS_Pre-update" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Pre-update"].mean()/ns_per_ms, 3)),
    "1D_SS_Conc_Load_Stream" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Conc_Load_Stream"].mean()/ns_per_ms, 3)),
    "1D_SS_Conc_Load_Stream_std" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Conc_Load_Stream"].std()/ns_per_ms, 3)),
    "1D_SS_Theoretical_Stream" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Theoretical_Stream"].mean()/ns_per_ms, 3)),
    "1D_SS_Stream_Latency" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Stream_Latency"].mean()/ns_per_ms, 3)),
    "1D_SS_Stream_Latency_std" : "{:.3f}".format(round(full_pickle_1D["data"]["S-Stream_Latency"].std()/ns_per_ms, 3)),
    "2D_path" : path_2D,
    "2D_title": full_pickle_2D["title"],
    "2D_Nt_x": full_pickle_2D["problem_definition"]["problem_params"]["Nt_x"],
    "2D_Nt_y" : full_pickle_2D["problem_definition"]["problem_params"]["Nt_y"],
    "2D_N_a1" : int(math.sqrt(full_pickle_2D["problem_definition"]["problem_params"]["num_target"])),
    "2D_N_a2" : int(math.sqrt(full_pickle_2D["problem_definition"]["problem_params"]["num_target"])),
    "2D_Nt1" : int(math.sqrt(full_pickle_2D["problem_definition"]["problem_params"]["num_target"])),
    "2D_Nt2" : int((full_pickle_2D["problem_definition"]["problem_params"]["Nt_x"] * 
                full_pickle_2D["problem_definition"]["problem_params"]["Nt_y"]) // 
                math.sqrt(full_pickle_2D["problem_definition"]["problem_params"]["num_target"])),
    "2D_roi_height" : full_pickle_2D["problem_definition"]["experiment_params"]["roi_height"],
    "2D_roi_width" : full_pickle_2D["problem_definition"]["experiment_params"]["roi_width"],
    "2D_IA_total_avg" : "{:.3f}".format(round(full_pickle_2D["data"]["I"].mean()/ns_per_ms, 3)),
    "2D_IA_total_std" : "{:.3f}".format(round(full_pickle_2D["data"]["I"].std()/ns_per_ms, 3)),
    "2D_IA_exposure" : "{:.3f}".format(round(IA_pred_dict_2D["T_exposure"])),
    "2D_IA_frame_transfer" : "{:.3f}".format(round(IA_pred_dict_2D["T_transfer"], 3)),
    "2D_IA_horz_shift" : "{:.3f}".format(round(IA_pred_dict_2D["T_horshift"] + IA_pred_dict_2D["T_register"], 3)),
    "2D_IA_DMA_transfer" : "{:.3f}".format(round(IA_pred_dict_2D["T_dma"] + full_pickle_2D["data"]["I"].mean()/ns_per_ms - IA_pred_dict_2D["T_total"], 3)),
    "2D_IA_latency" : "{:.3f}".format(round((full_pickle_2D["data"]["I"].mean()/ns_per_ms) - IA_pred_dict_2D["T_total"], 3)),
    "2D_IP_total_avg" : "{:.3f}".format(round(full_pickle_2D["data"]["S-IP_Total"].mean()/ns_per_ms, 3)),
    "2D_IP_total_std" : "{:.3f}".format(round(full_pickle_2D["data"]["S-IP_Total"].std()/ns_per_ms, 3)),
    "2D_IP_deconvolution" : "{:.3f}".format(round(full_pickle_2D["data"]["II-Deconvolution"].mean()/ns_per_ms, 3)),
    "2D_IP_threshold" : "{:.3f}".format(round(full_pickle_2D["data"]["II-Threshold"].mean()/ns_per_ms, 3)),
    "2D_PS_total_avg" : "{:.3f}".format(round(full_pickle_2D["data"]["III-Total"].mean()/ns_per_ms, 3)),
    "2D_PS_total_std" : "{:.3f}".format(round(full_pickle_2D["data"]["III-Total"].std()/ns_per_ms, 3)),
    "2D_PS_matching" : "{:.3f}".format(round(full_pickle_2D["data"]["III-Matching"].mean()/ns_per_ms, 3)),
    "2D_PS_batching" : "{:.3f}".format(round(full_pickle_2D["data"]["III-Batching"].mean()/ns_per_ms, 3)),
    "2D_SS_total_avg": "{:.3f}".format(round(full_pickle_2D["data"]["S-Synth_Load_Total"].mean()/ns_per_ms, 3)),
    "2D_SS_total_std": "{:.3f}".format(round(full_pickle_2D["data"]["S-Synth_Load_Total"].std()/ns_per_ms, 3)),
    "2D_SS_translate": "{:.3f}".format(round(full_pickle_2D["data"]["IV-Translate"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-lookup" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Pre-lookup"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-upload" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Pre-upload"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-update" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Pre-update"].mean()/ns_per_ms, 3)),
    "2D_SS_Conc_Load_Stream" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Conc_Load_Stream"].mean()/ns_per_ms, 3)),
    "2D_SS_Conc_Load_Stream_std" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Conc_Load_Stream"].std()/ns_per_ms, 3)),
    "2D_SS_Theoretical_Stream" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Theoretical_Stream"].mean()/ns_per_ms, 3)),
    "2D_SS_Stream_Latency" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Stream_Latency"].mean()/ns_per_ms, 3)),
    "2D_SS_Stream_Latency_std" : "{:.3f}".format(round(full_pickle_2D["data"]["S-Stream_Latency"].std()/ns_per_ms, 3)),
    "2D_path2" : path_2D2,
    "2D_title2": full_pickle_2D2["title"],
    "2D_Nt_x2": full_pickle_2D2["problem_definition"]["problem_params"]["Nt_x"],
    "2D_Nt_y2" : full_pickle_2D2["problem_definition"]["problem_params"]["Nt_y"],
    "2D_N_a12" : int(math.sqrt(full_pickle_2D2["problem_definition"]["problem_params"]["num_target"])),
    "2D_N_a22" : int(math.sqrt(full_pickle_2D2["problem_definition"]["problem_params"]["num_target"])),
    "2D_Nt12" : int(math.sqrt(full_pickle_2D2["problem_definition"]["problem_params"]["num_target"])),
    "2D_Nt22" : int((full_pickle_2D2["problem_definition"]["problem_params"]["Nt_x"] * 
                full_pickle_2D2["problem_definition"]["problem_params"]["Nt_y"]) // 
                math.sqrt(full_pickle_2D2["problem_definition"]["problem_params"]["num_target"])),
    "2D_roi_height2" : full_pickle_2D2["problem_definition"]["experiment_params"]["roi_height"],
    "2D_roi_width2" : full_pickle_2D2["problem_definition"]["experiment_params"]["roi_width"],
    "2D_n_alpha2" : 1,
    "2D_n_nu2" : 1,
    "2D_IA_total_avg2" : "{:.3f}".format(round(full_pickle_2D2["data"]["I"].mean()/ns_per_ms, 3)),
    "2D_IA_total_std2" : "{:.3f}".format(round(full_pickle_2D2["data"]["I"].std()/ns_per_ms, 3)),
    "2D_IA_exposure2" : "{:.3f}".format(round(IA_pred_dict_2D2["T_exposure"])),
    "2D_IA_frame_transfer2" : "{:.3f}".format(round(IA_pred_dict_2D2["T_transfer"], 3)),
    "2D_IA_horz_shift2" : "{:.3f}".format(round(IA_pred_dict_2D2["T_horshift"] + IA_pred_dict_2D2["T_register"], 3)),
    "2D_IA_DMA_transfer2" : "{:.3f}".format(round(IA_pred_dict_2D2["T_dma"] + full_pickle_2D2["data"]["I"].mean()/ns_per_ms - IA_pred_dict_2D2["T_total"], 3)),
    "2D_IA_latency2" : "{:.3f}".format(round((full_pickle_2D2["data"]["I"].mean()/ns_per_ms) - IA_pred_dict_2D2["T_total"], 3)),
    "2D_IP_total_avg2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-IP_Total"].mean()/ns_per_ms, 3)),
    "2D_IP_total_std2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-IP_Total"].std()/ns_per_ms, 3)),
    "2D_IP_deconvolution2" : "{:.3f}".format(round(full_pickle_2D2["data"]["II-Deconvolution"].mean()/ns_per_ms, 3)),
    "2D_IP_threshold2" : "{:.3f}".format(round(full_pickle_2D2["data"]["II-Threshold"].mean()/ns_per_ms, 3)),
    "2D_PS_total_avg2" : "{:.3f}".format(round(full_pickle_2D2["data"]["III-Total"].mean()/ns_per_ms, 3)),
    "2D_PS_total_std2" : "{:.3f}".format(round(full_pickle_2D2["data"]["III-Total"].std()/ns_per_ms, 3)),
    "2D_PS_matching2" : "{:.3f}".format(round(full_pickle_2D2["data"]["III-Matching"].mean()/ns_per_ms, 3)),
    "2D_PS_batching2" : "{:.3f}".format(round(full_pickle_2D2["data"]["III-Batching"].mean()/ns_per_ms, 3)),
    "2D_SS_total_avg2": "{:.3f}".format(round(full_pickle_2D2["data"]["S-Synth_Load_Total"].mean()/ns_per_ms, 3)),
    "2D_SS_total_std2": "{:.3f}".format(round(full_pickle_2D2["data"]["S-Synth_Load_Total"].std()/ns_per_ms, 3)),
    "2D_SS_translate2": "{:.3f}".format(round(full_pickle_2D2["data"]["IV-Translate"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-lookup2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Pre-lookup"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-upload2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Pre-upload"].mean()/ns_per_ms, 3)),
    "2D_SS_Pre-update2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Pre-update"].mean()/ns_per_ms, 3)),
    "2D_SS_Conc_Load_Stream2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Conc_Load_Stream"].mean()/ns_per_ms, 3)),
    "2D_SS_Conc_Load_Stream_std2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Conc_Load_Stream"].std()/ns_per_ms, 3)),
    "2D_SS_Theoretical_Stream2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Theoretical_Stream"].mean()/ns_per_ms, 3)),
    "2D_SS_Stream_Latency2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Stream_Latency"].mean()/ns_per_ms, 3)),
    "2D_SS_Stream_Latency_std2" : "{:.3f}".format(round(full_pickle_2D2["data"]["S-Stream_Latency"].std()/ns_per_ms, 3))
}

overall_avg_1D = float(variables["1D_IA_total_avg"]) +float(variables["1D_IP_total_avg"]) + float(variables["1D_PS_total_avg"]) + float(variables["1D_SS_total_avg"]) + float(variables["1D_SS_Conc_Load_Stream"])

variables["1D_overall_avg"] = "{:.3f}".format(round(overall_avg_1D, 3))

overall_std_1D = float(variables["1D_IA_total_std"]) +float(variables["1D_IP_total_std"]) + float(variables["1D_PS_total_std"]) + float(variables["1D_SS_total_std"]) + float(variables["1D_SS_Conc_Load_Stream_std"])

variables["1D_overall_std"] = "{:.3f}".format(round(overall_std_1D, 3))

overall_avg_2D = float(variables["2D_IA_total_avg"]) +float(variables["2D_IP_total_avg"]) + float(variables["2D_PS_total_avg"]) + float(variables["2D_SS_total_avg"]) + float(variables["2D_SS_Conc_Load_Stream"])

variables["2D_overall_avg"] = "{:.3f}".format(round(overall_avg_2D, 3))

overall_std_2D = float(variables["2D_IA_total_std"]) +float(variables["2D_IP_total_std"]) + float(variables["2D_PS_total_std"]) + float(variables["2D_SS_total_std"]) + float(variables["2D_SS_Conc_Load_Stream_std"])

variables["2D_overall_std"] = "{:.3f}".format(round(overall_std_2D, 3))

overall_avg_2D2 = float(variables["2D_IA_total_avg2"]) +float(variables["2D_IP_total_avg2"]) + float(variables["2D_PS_total_avg2"]) + float(variables["2D_SS_total_avg2"]) + float(variables["2D_SS_Conc_Load_Stream2"])

variables["2D_overall_avg2"] = "{:.3f}".format(round(overall_avg_2D2, 3))

overall_std_2D2 = float(variables["2D_IA_total_std2"]) +float(variables["2D_IP_total_std2"]) + float(variables["2D_PS_total_std2"]) + float(variables["2D_SS_total_std2"]) + float(variables["2D_SS_Conc_Load_Stream_std2"])

variables["2D_overall_std2"] = "{:.3f}".format(round(overall_std_2D2, 3))

# Replace placeholders with actual values
updated_latex_code = replace_variables_in_template(template_path, variables)

# Compile the updated LaTeX code into a PDF file
compile_latex(updated_latex_code, output_path)
print('Result compiled to', output_path)


1024
16
{'T_exposure': 20.0, 'T_transfer': 4.49887, 'T_horshift': 6.618453333333333, 'T_register': 0.03573333333333333, 'T_link': 0, 'T_dma': 0.31261657657657654, 'T_total': 31.465673243243245}
1024
128
{'T_exposure': 20.0, 'T_transfer': 4.49887, 'T_horshift': 10.441386666666666, 'T_register': 0.03573333333333333, 'T_link': 0, 'T_dma': 0.4779326126126126, 'T_total': 35.45392261261262}
1024
128
{'T_exposure': 20.0, 'T_transfer': 4.49887, 'T_horshift': 10.441386666666666, 'T_register': 0.03573333333333333, 'T_link': 0, 'T_dma': 0.4779326126126126, 'T_total': 35.45392261261262}


NameError: name 'run' is not defined

In [ ]:
print(IA_pred_dict_2D)
